In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from keras.models import load_model
from tensorflow import keras
model = keras.models.load_model('multi_digits_model_0_9.h5')

In [356]:
def display(img, n="img"):
    a = 0
#     cv2.imshow(n, img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    
def display_(img, n="img"):
    cv2.imshow(n, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [4]:
import random as rng
import cv2
def thresh_callback(val, src_gray, area_threshold = 35):
    threshold = val
    
    canny_output = cv2.Canny(src_gray, threshold, threshold * 2)
    display(canny_output)
    
    contours, _ = cv2.findContours(canny_output, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    
    
    contours_poly = [None]*len(contours)
    boundRect = [None]*len(contours)
    centers = [None]*len(contours)
    radius = [None]*len(contours)
    for i, c in enumerate(contours):
        contours_poly[i] = cv2.approxPolyDP(c, 3, True)
        boundRect[i] = cv2.boundingRect(contours_poly[i])
        centers[i], radius[i] = cv2.minEnclosingCircle(contours_poly[i])
    
    
    drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
    drawing = cv2.cvtColor(src_gray,cv2.COLOR_GRAY2BGR)
    print(len(contours))
    
    for i in range(len(contours)):
        print(i)
        color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
        x,y,w,h = boundRect[i]
        print("Area:",w*h)
        if w*h > area_threshold:
            cv2.rectangle(drawing, (int(boundRect[i][0]), int(boundRect[i][1])), \
              (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), color, 0)
        display(drawing)
#         cv2.drawContours(drawing, contours_poly, i, color)
#         if a < b:
#         cv2.circle(drawing, (int(centers[i][0]), int(centers[i][1])), int(radius[i]), color, 2)
    
    
#     cv.imshow('Contours', drawing)
    display(drawing)
    cv2.imwrite('out/30.png', drawing)
    return contours, boundRect

In [5]:
def preprocessing(im, border=10):
#     im = cv2.imread('../second/cells/'+i,0)
    _,th1 = cv2.threshold(im, 127,255, cv2.THRESH_OTSU)
#     plt.imshow(255-th1,cmap='gray');
    img = 255-th1
    h,w = img.shape
    mask = np.ones(img.shape[:2], np.uint8)*0
    mt = border
    mask[mt:h-mt, mt:w-mt] = 255
    # display(mask)
    img = cv2.bitwise_and(img,img,mask = mask)
#     display(img)
    return img

In [6]:
def contours_(img):
    mser = cv2.MSER_create()

    #Your image path i-e receipt path
    # img = cv2.imread('svhn-data/train_answer/3.jpg')

    #Convert to gray scale
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    vis = img.copy()

    #detect regions in gray scale image
    regions, _ = mser.detectRegions(gray)

    hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions]

    cv2.polylines(vis, hulls, 1, (0, 255, 0))

    display(vis)
    mask = np.zeros((img.shape[0], img.shape[1], 1), dtype=np.uint8)

    for contour in hulls:

        cv2.drawContours(mask, [contour], -1, (255, 255, 255), -1)

    #this is used to find only text regions, remaining are ignored
    text_only = cv2.bitwise_and(img, img, mask=mask)



In [185]:
img = cv2.imread('../second/cells/img509_30.5_.jpg',0)
# img = cv2.resize(img, (28,28))
# t = np.mean(img) + np.std(img)/2
t = 150
# img[img < t] = 0
# img[img > t] = 255
img = preprocessing(img)
vis = cv2.Canny(img,1,0)
display(img)
display(vis)
mser = cv2.MSER_create()
regions = mser.detectRegions(vis)
hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions[0]]
mser = cv2.MSER_create()
regions, _ = mser.detectRegions(vis)
print(len(regions))
for p in regions:
    xmax, ymax = np.amax(p, axis=0)
    xmin, ymin = np.amin(p, axis=0)
    cv2.rectangle(vis, (xmin,ymax), (xmax,ymin), (150), 1)
#     display(vis)

display(vis)
# vis = cv2.resize(vis, (28,28))
# display(vis)
cv2.polylines(vis, hulls, 10, (125))
cv2.imwrite('/tmp/tmp1.png', vis)
display(vis)

3


In [186]:
#opencv
canny_output = cv2.Canny(img,1,0)
contours, hierarchy = cv2.findContours(canny_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# Draw contours
drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
for i in range(len(contours)):
    color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
    cv2.drawContours(drawing, contours, i, color, 2, cv2.LINE_8, hierarchy, 0)

## Main

In [428]:
#opencv
#finding rectangles
img = cv2.imread('../second/cells/img1200_1.5_.jpg',0)
img = preprocessing(img, border=5)
canny_output = cv2.Canny(img,1,0)
display(img)
display(canny_output)
contours, hierarchy = cv2.findContours(canny_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# Draw contours
drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
rec = []
for contour in contours:
    (x,y,w,h) = cv2.boundingRect(contour)
    rec.append((x,y,w,h))
    print(x,y,w,h)
    cv2.rectangle(drawing, (x,y), (x+w,y+h), (0,255,0), 2)
display(drawing)

31 21 8 9
31 21 8 9
44 14 10 13
44 14 10 13
18 12 9 23
49 7 3 3
49 7 3 3
53 4 14 4


In [429]:
rects = []
for contour in contours:
    (x,y,w,h) = cv2.boundingRect(contour)
    x1=x
    y1=y
    x2=x+w
    y2=y+h
    rects.append((x1,y1,x2,y2))
print(rects)
rects.sort(key=lambda x: x[0])
print(rects)
drawing = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
for r in rects:
    (x,y,x2,y2) = r
#     rec.append((x,y,w,h))
    print(x2-x,y2-y)
    cv2.rectangle(drawing, (x,y), (x2,y2), (0,255,0), 2)
    display(drawing)

[(31, 21, 39, 30), (31, 21, 39, 30), (44, 14, 54, 27), (44, 14, 54, 27), (18, 12, 27, 35), (49, 7, 52, 10), (49, 7, 52, 10), (53, 4, 67, 8)]
[(18, 12, 27, 35), (31, 21, 39, 30), (31, 21, 39, 30), (44, 14, 54, 27), (44, 14, 54, 27), (49, 7, 52, 10), (49, 7, 52, 10), (53, 4, 67, 8)]
9 23
8 9
8 9
10 13
10 13
3 3
3 3
14 4


In [430]:
def group_together(rects):
    p = 0
    grouped = []
    l = len(rects)
    i  = 0
    print(l)
    while i != len(rects)-1:
        
        (x1,y1,x2,y2) = rects[i]
        print("checking for:",rects[i], ":", i)
        (p1,q1,p2,q2) = rects[i+1]
#         if x1 <= p1 <= x2 and y1 <= q1 <= y2:
        if x1 <= p1 <= x2:# and y1 <= q1 <= y2:
            print('yes')
            xx1 = min(x1,p1)
            yy1 = min(y1,q1)
            xx2 = max(x2,p2)
            yy2 = max(y2,q2)
            print("previous:",rects[i])
            rects[i] = (xx1,yy1,xx2,yy2)
            print("new:",rects[i])
            print("deleting:",rects[i+1])
            rects.pop(i+1)
        else:    
            i = i+1
#         print("O ")
    display(img)
    drawing = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    for r in rects:
        (x,y,x2,y2) = r
    #     rec.append((x,y,w,h))
        print(x2-x,y2-y)
        cv2.rectangle(drawing, (x,y), (x2,y2), (0,255,0), 2)
        display(drawing)
group_together(rects)

8
checking for: (18, 12, 27, 35) : 0
checking for: (31, 21, 39, 30) : 1
yes
previous: (31, 21, 39, 30)
new: (31, 21, 39, 30)
deleting: (31, 21, 39, 30)
checking for: (31, 21, 39, 30) : 1
checking for: (44, 14, 54, 27) : 2
yes
previous: (44, 14, 54, 27)
new: (44, 14, 54, 27)
deleting: (44, 14, 54, 27)
checking for: (44, 14, 54, 27) : 2
yes
previous: (44, 14, 54, 27)
new: (44, 7, 54, 27)
deleting: (49, 7, 52, 10)
checking for: (44, 7, 54, 27) : 2
yes
previous: (44, 7, 54, 27)
new: (44, 7, 54, 27)
deleting: (49, 7, 52, 10)
checking for: (44, 7, 54, 27) : 2
yes
previous: (44, 7, 54, 27)
new: (44, 4, 67, 27)
deleting: (53, 4, 67, 8)
9 23
8 9
23 23


In [431]:
def pad_to_square(rects):
    imgs = []
    h_, w_ = 32,32
    
    for i in rects:
        (x,y,x2,y2) = i
        zero = np.zeros((h_,w_))
        ver_pad = w_ - (y2-y)
        hor_pad = h_ - (x2-x)
        top_pad = ver_pad // 2
        bot_pad = ver_pad - top_pad
        left_pad = hor_pad // 2
        right_pad = hor_pad - left_pad
#         print(64 - right_pad - left_pad)
#         print(64 - top_pad - bot_pad)
        print(x2-x,y2-y)
        print(w_ - right_pad - left_pad,h_-bot_pad - top_pad)
        display(img[y:y2,x:x2])
        zero[top_pad:h_- bot_pad, left_pad:w_ - right_pad] = img[y:y2,x:x2]
        display(zero)
#         print(zero.shape)
        zero = cv2.resize(zero, (28,28))
        display(zero)
        imgs.append(zero)
        print("Done with:", i)
    return imgs
display(img)
imgs = pad_to_square(rects)

9 23
9 23
Done with: (18, 12, 27, 35)
8 9
8 9
Done with: (31, 21, 39, 30)
23 23
23 23
Done with: (44, 4, 67, 27)


In [432]:
imgs_list = imgs
imgs = np.array(imgs)
imgs = np.reshape(imgs,(imgs.shape[0] ,28,28,1))
imgs[0].shape

(28, 28, 1)

In [433]:
pred = model.predict(imgs)

In [434]:
for i in range(len(pred)): 
    print(pred[i].argmax())
    display_(imgs_list[i], str(pred[i].argmax()))

1
5
3


In [427]:
dots.append(imgs_list[1])

In [391]:
c = 0
for i in dots:
    cv2.imwrite("out/zero"+str(c)+"_._.png",i)
    print(i.shape)
    c += 1

(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)


In [52]:
for i in dots:
    display(i)

In [20]:
dots = []

In [64]:
dots.pop(0)

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  18.21428653,  18.21428653,
         11.70917427,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,  35.12757615,
         54.64286149,  54.64286149, 255.        , 255.        ,
        183.44376992,  54.64286149,  19.51533745,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   

In [ ]:
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#opencv
canny_output = cv2.Canny(img,1,0)
contours, hierarchy = cv2.findContours(canny_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# Draw contours
drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
for i in range(len(contours)):
    color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
    cv2.drawContours(drawing, contours, i, color, 2, cv2.LINE_8, hierarchy, 0)

In [ ]:
#opencv
canny_output = cv2.Canny(img,1,0)
contours, hierarchy = cv2.findContours(canny_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# Draw contours
drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
for i in range(len(contours)):
    color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
    cv2.drawContours(drawing, contours, i, color, 2, cv2.LINE_8, hierarchy, 0)

In [ ]:
#opencv
canny_output = cv2.Canny(img,1,0)
contours, hierarchy = cv2.findContours(canny_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# Draw contours
drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
for i in range(len(contours)):
    color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
    cv2.drawContours(drawing, contours, i, color, 2, cv2.LINE_8, hierarchy, 0)

In [ ]:
#opencv
canny_output = cv2.Canny(img,1,0)
contours, hierarchy = cv2.findContours(canny_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# Draw contours
drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
for i in range(len(contours)):
    color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
    cv2.drawContours(drawing, contours, i, color, 2, cv2.LINE_8, hierarchy, 0)

In [ ]:
ret, thresh = cv2.threshold(img, 127, 255, 0)
im2, contours = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
con_img = img.copy()
cv2.drawContours(con_img, contours, -1, (0,255,0), 3)
display(con_img)

In [ ]:
# kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 1))
# connected = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
# display(connected)
# find all the contours
contours, hierarchy,=cv2.findContours(img.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
#Segment the text lines
counter=0
array_of_texts=[]
for idx in range(len(contours)):
    x, y, w, h = cv2.boundingRect(contours[idx])
    cropped_image = img[x-10:x+w+10,y: y+h ]
#     str_store = re.sub(r'([^\s\w]|_)+', '', image_to_string(cropped_image))
#     array_of_texts.append(str_store)
#     counter+=1
    print(cropped_image.shape)
    display(cropped_image)

print(array_of_texts)

In [ ]:
im = cv2.imread('../second/cells/img1109_8.5_.jpg',0)
preprocessing(im)

In [ ]:
contours_(im)

In [ ]:
f = os.listdir("../second/cells/")
c = 0
a = 0
for i in f:
    if i != "empty":
        lab = i.split('_')
        im = cv2.imread('../second/cells/'+i,0)
        _,th1 = cv2.threshold(im, 127,255, cv2.THRESH_OTSU)
    #     plt.imshow(255-th1,cmap='gray');
        img = 255-th1
        h,w = img.shape
        mask = np.ones(img.shape[:2], np.uint8)*0
        mt = 10
        mask[mt:h-mt, mt:w-mt] = 255
        # display(mask)
        img = cv2.bitwise_and(img,img,mask = mask)
        plt.imshow(img);
#         img = cv2.resize(img,(64,64))
#         z = np.zeros((64,64))
#         # z[16:48,16:48] = img
#         z = img
#         x_img = np.reshape(z,(1,64,64,1))
#         test_pred = session.run(y_pred_cls, feed_dict={x: x_img, keep_prob: 1.0})
#         pred_number = ''.join(str(x) for x in test_pred[0] if x != 10)
#         print(pred_number,lab[1], float(pred_number)==float(lab[1]))
#         if float(pred_number)==float(lab[1]):
#             a += 1
#         c += 1
# print("Accuracy: ",a/c)
#         plt.imshow(img,cmap='gray');